# **Step 1: Initialize Spark Session**
First, set up your Spark environment to handle large-scale data processing:

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=cd38c431d7bd479d56f1552dab8857aa72a2aefbb1b322cc59983f5466f19acc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Sentiment Detection") \
    .master("local[*]") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "10g") \
    .config("spark.default.parallelism", "100") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrationRequired", "true") \
    .config("spark.kryoserializer.buffer.max", "1024m") \
    .getOrCreate()


# Step 2: Load the data
Read the dataset from a file, which could be stored on your Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Adding schematiuc view while loading to speaden up the process rather than inferring a schema

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema = StructType([
    StructField("customer_id", StringType(), True),  # Change to IntegerType if applicable
    StructField("product_id", StringType(), True),
    StructField("product_parent", StringType(), True),
    StructField("product_title", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("star_rating", IntegerType(), True),
    StructField("full_text", StringType(), True)
])

In [ ]:
# Path to your dataset
data_path = '/content/drive/MyDrive/Big Data Project/language.parquet/'

# Load the dataset
df = spark.read.parquet(data_path)

In [ ]:
df.show(n=200)

+-----------+----------+--------------+--------------------+--------------------+-----------+--------------------+--------+
|customer_id|product_id|product_parent|       product_title|    product_category|star_rating|           full_text|language|
+-----------+----------+--------------+--------------------+--------------------+-----------+--------------------+--------+
|   45694511|B002ACCC52|     579166665|Delta Children Ec...|                Baby|          5|Five Stars. Easy ...|      en|
|   38614677|B00H9L7VIW|     851045898|boostULTIMATE - 6...|Personal_Care_App...|          5| It works.. good sex|      en|
|   49760248|B00934G9Y0|     315666168|Safco Products 47...|           Furniture|          5|Awesome Chair!. T...|      en|
|   11152323|B00KYXX0H8|     242677748|VIZIO Qwerty Remo...|         Electronics|          1|Dont see that the...|      en|
|    9591120|B00719R8RG|     892385878|North American Be...|                Baby|          3|Small but cute. A...|      en|
|   1206

In [ ]:
df = df.repartition(200)

# Step 3 : Sentiment Analysis

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

models = {
    "en": "nlptown/bert-base-multilingual-uncased-sentiment",
    "es": "dccuchile/bert-base-spanish-wwm-cased",
    "fr": "camembert/camembert-large",
    "de": "bert-base-german-dbmdz-uncased",
    "zh": "bert-base-chinese",
    "ar": "aubmindlab/bert-base-arabert",
    "ru": "DeepPavlov/rubert-base-cased",
    "pt": "neuralmind/bert-base-portuguese-cased",
    "nl": "wietsedv/bert-base-dutch-cased",
    "it": "dbmdz/bert-base-italian-xxl-cased"
}

# Load models and tokenizers outside UDF
model_objects = {}
for lang, model_path in models.items():
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model_objects[lang] = (model, tokenizer)

@pandas_udf("integer", PandasUDFType.SCALAR)
def get_sentiment(texts: pd.Series, lang_codes: pd.Series) -> pd.Series:
    results = []
    for text, lang_code in zip(texts, lang_codes):
        model, tokenizer = model_objects.get(lang_code, model_objects["en"])
        inputs = tokenizer(list(texts), return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_labels = predictions.argmax(dim=1)
        results.extend(predicted_labels.tolist())
    return pd.Series(results)

df = df.withColumn("sentiment", get_sentiment(col("full_text"), col("language")))
df.cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-xxl-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


DataFrame[customer_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: bigint, full_text: string, language: string, sentiment: int]

In [ ]:
# Continue processing or save the results
df.write.format("parquet").save("/content/drive/MyDrive/Big Data Project/sentiment.parquet", mode='overwrite')